In [1]:
import os

In [2]:
#creating base folders
base_dir = 'D:\\datasets\\catsNdogs_large_dataset\\PetImages'
def Create_base_dirs(dir)->None:
    '''function docstring'''
    os.mkdir(f'{dir}\\ready_to_gen')
    os.mkdir(f'{dir}\\ready_to_gen\\training')
    os.mkdir(f'{dir}\\ready_to_gen\\training\\cat')
    os.mkdir(f'{dir}\\ready_to_gen\\training\\dog')
    os.mkdir(f'{dir}\\ready_to_gen\\validation')
    os.mkdir(f'{dir}\\ready_to_gen\\validation\\cat')
    os.mkdir(f'{dir}\\ready_to_gen\\validation\\dog')
try :
    Create_base_dirs(base_dir)
except :
    print('something wrong when running the function')

In [4]:
#splitting data
from random import sample
from shutil import copyfile
from tensorflow.keras.preprocessing.image import load_img
cats_dir = 'D:\\datasets\\catsNdogs_large_dataset\\PetImages\\Cat'
dogs_dir = 'D:\\datasets\\catsNdogs_large_dataset\\PetImages\\Dog'
cats_dir_tr = 'D:\\datasets\\catsNdogs_large_dataset\\PetImages\\ready_to_gen\\training\\cat'
dogs_dir_tr = 'D:\\datasets\\catsNdogs_large_dataset\\PetImages\\ready_to_gen\\training\\dog'
cats_dir_valid = 'D:\\datasets\\catsNdogs_large_dataset\\PetImages\\ready_to_gen\\validation\\cat'
dogs_dir_valid = 'D:\\datasets\\catsNdogs_large_dataset\\PetImages\\ready_to_gen\\validation\\dog'
def Split_data(base_dir,train_dir,valid_dir,split_size)->None :
    '''function docstring'''
    split = (split_size*len(os.listdir(base_dir))) // 100
    selected_pictures = sample(os.listdir(base_dir),split)
    for pic in os.listdir(base_dir) :
        try :
            test = load_img(f'{base_dir}\\{pic}')
            if pic in selected_pictures :
                copyfile(f'{base_dir}\\{pic}',f'{train_dir}\\{pic}')
            else :
                copyfile(f'{base_dir}\\{pic}',f'{valid_dir}\\{pic}')
        except :
            print('%s is zero length , so ignoring'%(pic))
try :
    Split_data(cats_dir,cats_dir_tr,cats_dir_valid,90)
    Split_data(dogs_dir,dogs_dir_tr,dogs_dir_valid,90)
except :
    print('something wrong when running the function')
print('Splitting done')

666.jpg is zero length , so ignoring
Thumbs.db is zero length , so ignoring
11702.jpg is zero length , so ignoring


C:\Users\USER\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:845: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Thumbs.db is zero length , so ignoring
Splitting done


In [4]:
#creating generator instances
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(rescale=1/255,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,horizontal_flip=True,zoom_range=0.2,)
valid_gen = ImageDataGenerator(rescale=1/255)

In [5]:
#generating dataset
ready_train = train_gen.flow_from_directory('D:\\datasets\\catsNdogs_large_dataset\\PetImages\\ready_to_gen\\training',target_size=(150,150),batch_size=224,class_mode='binary')
ready_valid = valid_gen.flow_from_directory('D:\\datasets\\catsNdogs_large_dataset\\PetImages\\ready_to_gen\\validation',target_size=(150,150),batch_size=25,class_mode='binary')

Found 22496 images belonging to 2 classes.
Found 2502 images belonging to 2 classes.


In [42]:
#creating a model
from tensorflow.keras import models , layers , Model
from tensorflow.keras.applications import InceptionV3
m = Model()
def Create_model() :
    pre_trained_model = InceptionV3(include_top=False,input_shape=(150,150,3))
    for layer in pre_trained_model.layers :
        layer.trainable = False
    last_layer = pre_trained_model.get_layer('mixed7')
    last_out = last_layer.output
    x = layers.Flatten()(last_out)
    x = layers.Dense(256,activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1,activation='sigmoid')(x)
    model = Model(pre_trained_model.input,x)
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model
try :
    Create_model()
    print('model has created')
except :
    print('something wrong when running the function')

model has created


In [9]:
#model training
history = Create_model().fit(ready_train,epochs=15,validation_data= ready_valid)

Epoch 1/15
101/101 [==============================] - 571s 6s/step - loss: 0.6064 - accuracy: 0.8626 - val_loss: 0.1105 - val_accuracy: 0.9524
Epoch 2/15
101/101 [==============================] - 241s 2s/step - loss: 0.1811 - accuracy: 0.9267 - val_loss: 0.0922 - val_accuracy: 0.9616
Epoch 3/15
101/101 [==============================] - 166s 2s/step - loss: 0.1489 - accuracy: 0.9393 - val_loss: 0.1025 - val_accuracy: 0.9620
Epoch 4/15
101/101 [==============================] - 167s 2s/step - loss: 0.1430 - accuracy: 0.9415 - val_loss: 0.0947 - val_accuracy: 0.9664
Epoch 5/15
101/101 [==============================] - 228s 2s/step - loss: 0.1378 - accuracy: 0.9456 - val_loss: 0.0996 - val_accuracy: 0.9644
Epoch 6/15
101/101 [==============================] - 220s 2s/step - loss: 0.1301 - accuracy: 0.9470 - val_loss: 0.0888 - val_accuracy: 0.9680
Epoch 7/15
101/101 [==============================] - 202s 2s/step - loss: 0.1298 - accuracy: 0.9483 - val_loss: 0.0925 - val_accuracy: 0.9608

In [14]:
#storing results in a dictionary
training_results = history.history
for acc in training_results['accuracy'] :
    print('%0.2f'%(acc*100))

86.26
92.67
93.93
94.15
94.56
94.70
94.83
94.69
94.94
94.98
95.34
95.34
95.43
95.20
95.49
